In [23]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O 
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.metrics import classification_report
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout, BatchNormalization
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.optimizers import Adam
import os
import tkinter as tk
from tkinter import filedialog
from tkinter.scrolledtext import ScrolledText

In [24]:

ddos_data_test=[]
ddos_data_test_copy=[]
non_binary_columns=[]

In [25]:
final_columns = [' Source IP', ' Source Port', ' Destination IP', ' Destination Port',
       ' Protocol', 'Total Length of Fwd Packets', ' Fwd Packet Length Min',
       ' Bwd Packet Length Min', ' Flow IAT Min', 'Bwd IAT Total',
       ' Bwd IAT Mean', ' Bwd Header Length', 'Fwd Packets/s',
       ' Bwd Packets/s', ' Min Packet Length', ' Packet Length Mean',
       'FIN Flag Count', ' SYN Flag Count', ' PSH Flag Count',
       ' ACK Flag Count', ' URG Flag Count', ' ECE Flag Count',
       ' Down/Up Ratio', ' Avg Fwd Segment Size', ' Avg Bwd Segment Size',
       'Init_Win_bytes_forward', ' Init_Win_bytes_backward',
       ' act_data_pkt_fwd', ' min_seg_size_forward', ' Active Std',
       ' Active Min', ' Idle Std', ' Idle Min', ' Label']

In [26]:
train_columns =['Flow ID', ' Source IP',' Source Port', ' Destination IP',' Destination Port',' Protocol',' Timestamp',
 ' Flow Duration',' Total Fwd Packets',' Total Backward Packets','Total Length of Fwd Packets', ' Total Length of Bwd Packets',
' Fwd Packet Length Max', ' Fwd Packet Length Min', ' Fwd Packet Length Mean', ' Fwd Packet Length Std', 'Bwd Packet Length Max',
 ' Bwd Packet Length Min', ' Bwd Packet Length Mean', ' Bwd Packet Length Std', 'Flow Bytes/s', ' Flow Packets/s', ' Flow IAT Mean',
 ' Flow IAT Std', ' Flow IAT Max', ' Flow IAT Min', 'Fwd IAT Total', ' Fwd IAT Mean', ' Fwd IAT Std', ' Fwd IAT Max', ' Fwd IAT Min',
 'Bwd IAT Total', ' Bwd IAT Mean', ' Bwd IAT Std', ' Bwd IAT Max', ' Bwd IAT Min', 'Fwd PSH Flags', ' Bwd PSH Flags', ' Fwd URG Flags',
 ' Bwd URG Flags', ' Fwd Header Length', ' Bwd Header Length', 'Fwd Packets/s', ' Bwd Packets/s', ' Min Packet Length', ' Max Packet Length',
 ' Packet Length Mean', ' Packet Length Std', ' Packet Length Variance', 'FIN Flag Count', ' SYN Flag Count', ' RST Flag Count', ' PSH Flag Count',
 ' ACK Flag Count', ' URG Flag Count', ' CWE Flag Count', ' ECE Flag Count', ' Down/Up Ratio', ' Average Packet Size', ' Avg Fwd Segment Size',
 ' Avg Bwd Segment Size', 'Fwd Avg Bytes/Bulk', ' Fwd Avg Packets/Bulk', ' Fwd Avg Bulk Rate', ' Bwd Avg Bytes/Bulk',
 ' Bwd Avg Packets/Bulk', 'Bwd Avg Bulk Rate', 'Subflow Fwd Packets', ' Subflow Fwd Bytes', ' Subflow Bwd Packets', ' Subflow Bwd Bytes',
 'Init_Win_bytes_forward', ' Init_Win_bytes_backward', ' act_data_pkt_fwd', ' min_seg_size_forward', 'Active Mean', ' Active Std', ' Active Max',
 ' Active Min', 'Idle Mean', ' Idle Std', ' Idle Max', ' Idle Min', ' Label']


In [27]:
constant_columns = [' Bwd PSH Flags', ' Fwd URG Flags', ' Bwd URG Flags', ' CWE Flag Count',
       'Fwd Avg Bytes/Bulk', ' Fwd Avg Packets/Bulk', ' Fwd Avg Bulk Rate',
       ' Bwd Avg Bytes/Bulk', ' Bwd Avg Packets/Bulk', 'Bwd Avg Bulk Rate']

# functions definition:

In [28]:
def normalize_timestamp(ddos_data):
    try:
        ddos_data[' Timestamp'] = pd.to_datetime(ddos_data[' Timestamp'])
        ddos_data.sort_values(by=' Timestamp', inplace=True)
    except Exception as e:
        print(f"Une erreur s'est produite lors de la normalisation des horodatages : {str(e)}")


label_encoder=LabelEncoder()
def Encode_Label_Column_1(ddos_data):
    label_mapping = {'BENIGN': 0, 'Syn': 1, 'TFTP': 1, 'DrDoS_LDAP': 1, 'DrDoS_UDP': 1, 'UDP-lag': 1,
                 'DrDoS_SNMP': 1, 'DrDoS_SSDP': 1, 'DrDoS_NetBIOS': 1, 'DrDoS_MSSQL': 1,
                 'DrDoS_DNS': 1, 'DrDoS_NTP': 1, 'WebDDoS': 1}
    #ddos_data[' Label']=label_encoder.fit_transform(ddos_data[' Label'])
    ddos_data[' Label'] = ddos_data[' Label'].map(label_mapping)


def dropping_const_and_red_columns(ddos_data):
    try:
        ddos_data.drop(['Flow ID',],axis=1,inplace=True)
        ddos_data.drop(columns=constant_columns,inplace=True)
    except Exception as e:
        print(f"Une erreur s'est produite lors de la suppression des colonnes : {str(e)}")

def encoding_ip_srv_dst(ddos_data):
    all_ip=pd.concat([ddos_data[' Source IP'],ddos_data[' Destination IP']])
    label_encoder.fit(all_ip)
    ddos_data[' Source IP']=label_encoder.transform(ddos_data[' Source IP'])
    ddos_data[' Destination IP']=label_encoder.transform(ddos_data[' Destination IP'])

def setting_time_as_an_index(ddos_data):
    ddos_data.set_index(' Timestamp', inplace=True)


scaler=StandardScaler()
def scale_num_features(ddos_data):
    global non_binary_columns
    ddos_data[non_binary_columns]=scaler.fit_transform(ddos_data[non_binary_columns])

def data_preprocessing(ddos_data_test):
    global non_binary_columns
    try:
        ddos_data_test = ddos_data_test[train_columns]
        #event_log.insert(tk.END, "normalization...\n")
        normalize_timestamp(ddos_data_test)
        #event_log.insert(tk.END, "columns cleaning...\n")
        dropping_const_and_red_columns(ddos_data_test)
        #event_log.insert(tk.END, "Label Culomn enconding...\n")
        Encode_Label_Column_1(ddos_data_test)
        #event_log.insert(tk.END, "ip_srv_dst Culomn enconding...\n")
        encoding_ip_srv_dst(ddos_data_test)
        setting_time_as_an_index(ddos_data_test)
        ddos_data_test=ddos_data_test[final_columns]
        non_binary_columns=ddos_data_test.columns[ddos_data_test.nunique()>2]
        scale_num_features(ddos_data_test)
        return (ddos_data_test)
    except Exception as e:
        print(f"Une erreur s'est produite lors du preprocssing : {str(e)}")


def data_splitting(ddos_data_test):
    try:
        
        n_future = 1
        n_past = 10
        
        # Préallouer les tableaux NumPy
        num_samples = len(ddos_data_test) - n_future - n_past + 1
        testX = np.zeros((num_samples, n_past, ddos_data_test.shape[1]))
        testY = np.zeros((num_samples,))
        
        for i in range(n_past, len(ddos_data_test) - n_future + 1):
            # Remplir les tableaux NumPy
            testX[i - n_past] = ddos_data_test.iloc[i - n_past:i, :].values
            testY[i - n_past] = ddos_data_test.iloc[i + n_future - 1, ddos_data_test.columns.get_loc(' Label')]
        
        return testX,testY 
    except Exception as e:
        print(f"Une erreur s'est produite lors du splitting : {str(e)}")      


# Model deployment

In [29]:

def open_csv_file():
    global ddos_data_test
    global ddos_data_test_copy
    try:
        file_path = filedialog.askopenfilename(filetypes=[("CSV files", "*.csv")])
        if file_path:
            file_name = os.path.basename(file_path)
            ddos_data_test = pd.read_csv(file_path)
            if (' Fwd Header Length.1' in ddos_data_test.columns):
                ddos_data_test = ddos_data_test.drop(columns=[' Fwd Header Length.1'])
            ddos_data_test.columns=train_columns
            ddos_data_test_copy=ddos_data_test
            event_log.insert(tk.END, f"csv file opened : {file_name}\n")
    except Exception as e:
        print(f"Une erreur s'est produite lors de l'ouverture du fichier CSV : {str(e)}")



def start_model_execution():
    global ddos_data_test
    global ddos_data_test_copy
    global execution_en_cours
    
    # Liste pour stocker les IP des connexions détectées comme des attaques
    ip_attaques = []
    
    try:
      
        event_log.insert(tk.END, "Data preprocessing...\n")
        ddos_data_test = data_preprocessing(ddos_data_test)
        testX, testY = data_splitting(ddos_data_test)
        event_log.insert(tk.END, "model loading...\n")
        model = tf.keras.models.load_model('ddos.h5')
        event_log.insert(tk.END, "attack detection...\n")
        y_pred = model.predict(testX)
        binary_predictions = (y_pred > 0.5).astype(int)
        j=0
        for i in range(len(binary_predictions)):
            if binary_predictions[i] == 1:  # Vérifiez si la prédiction est une attaque
                j+=1
                ip_attack = ddos_data_test_copy.iloc[i, 0]
                ip_attaques.append(ip_attack)  # Ajoutez l'IP à la liste des attaques
        print(f"{j} connexions / {len(binary_predictions)} sont des attaques")
        event_log.insert(tk.END,f"{j} connexions / {len(binary_predictions)} sont des attaques")
        
        # Conversion de la liste en DataFrame pandas
        df_attaques = pd.DataFrame(ip_attaques, columns=['IP_attack'])
        
        # Sauvegarde du DataFrame dans un fichier CSV
        df_attaques.to_csv('attaques_detectees.csv', index=False)
        
        
    except Exception as e:
        print(f"Une erreur s'est produite lors de l'exécution du modèle : {str(e)}")


def open_prediction_report():
    try:
        attack_report = pd.read_csv("C:/Users/NGESSI/Documents/DDoS Prediction/v2/attaques_detectees.csv")
        event_log.insert(tk.END, f"csv file opened : {attack_report}\n")
    except Exception as e:
        print(f"Une erreur s'est produite lors de l'ouverture du rapport  : {str(e)}")

def clear_event_log():
    event_log.delete('1.0', tk.END)  

root = tk.Tk()
root.title("NETAD 1.0")

file_start_frame = tk.Frame(root)
file_start_frame.pack(padx=30, pady=(10, 0), fill=tk.X)


open_button = tk.Button(file_start_frame, text="Choose data flow", command=open_csv_file)
open_button.grid(row=0, column=1, padx=(0, 5))

start_button = tk.Button(file_start_frame, text="Start detection", command=start_model_execution)
start_button.grid(row=0, column=2, padx=(0, 5))

open_button1 = tk.Button(file_start_frame, text="Open report", command=open_prediction_report)
open_button1.grid(row=0, column=3, padx=(0, 5))

clear_button = tk.Button(file_start_frame, text="Clear Event Log", command=clear_event_log)
clear_button.grid(row=0, column=4, padx=(0, 5))


event_frame = tk.Frame(root)
event_frame.pack(padx=10, pady=(0, 10), fill=tk.BOTH, expand=True)
event_log = ScrolledText(event_frame, width=60, height=10)
event_log.pack(padx=5, pady=5, fill=tk.BOTH, expand=True)

root.mainloop()


625/625 [==============================] - 5s 7ms/step
8632 connexions / 19990 sont des attaques
